<a href="https://colab.research.google.com/github/Chienstartup/ISIC_2024/blob/main/GBM_hypertuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install optuna
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.1 MB/s eta 0:00:00


In [21]:
import numpy as np
import pandas as pd
import optuna
import lightgbm as lgb
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, PredefinedSplit
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
import xgboost as xgb
from sklearn.ensemble import VotingClassifier
import catboost as cb

In [10]:
sampled_df = pd.read_csv('https://raw.githubusercontent.com/Chienstartup/ISIC_2024/main/meta_data/df_train_filtered.csv')

In [13]:
feature_cols = ['clin_size_long_diam_mm', 'tbp_lv_H_between_std', 'lesion_visibility_score', 'tbp_lv_deltaLBnorm_between_std',
 'age_approx_zscore', 'tbp_lv_B', 'avg_contrast', 'hue_contrast', 'anatom_site_general_encoded', 'attribution_encoded',
 'tbp_lv_C', 'tbp_lv_nevi_confidence_between_std', 'tbp_lv_B_between_std', 'tbp_lv_Aext']

target_col = 'target'

<h2>Competition: Partial AUC Metric

In [14]:
def custom_metric_binary(y_true, y_pred):
    y_hat = y_pred[:, 1] if y_pred.ndim > 1 else y_pred
    y_true_binary = y_true

    min_tpr = 0.80
    max_fpr = 1 - min_tpr

    v_gt = 1 - y_true_binary
    v_pred = 1 - y_hat

    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)

    return partial_auc

<h2> LGB Hypertuning

In [15]:
# def lgb_objective(trial):
#     params = {
#         'objective': 'binary',
#         'num_class': 1,
#         'n_estimators': 200,
#         'verbosity': -1,
#         'boosting_type': 'gbdt',
#         'lambda_l1': trial.suggest_float('lambda_l1', 1e-3, 10.0, log=True),
#         'lambda_l2': trial.suggest_float('lambda_l2', 1e-3, 10.0, log=True),
#         'learning_rate': trial.suggest_float('learning_rate', 1e-2, 1e-1, log=True),
#         'max_depth': trial.suggest_int('max_depth', 4, 8),
#         'num_leaves': trial.suggest_int('num_leaves', 16, 256),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
#         'colsample_bynode': trial.suggest_float('colsample_bynode', 0.4, 1.0),
#         'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
#         'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
#         'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
#         'scale_pos_weight' : trial.suggest_float('scale_pos_weight', 0.8, 4.0),
#         'device': 'cpu'
#     }

#     X = sampled_df[feature_cols]
#     y = sampled_df[target_col]
#     fold = sampled_df['fold']

#     ps = PredefinedSplit(fold)

#     cv_scores = []
#     for train_index, val_index in ps.split():
#         X_train, X_val = X.iloc[train_index], X.iloc[val_index]
#         y_train, y_val = y.iloc[train_index], y.iloc[val_index]

#         model = lgb.LGBMClassifier(**params, random_state=42)
# #         model = lgb.LGBMClassifier(**params)
#         model.fit(X_train, y_train)

#         y_pred = model.predict_proba(X_val)
#         score = custom_metric_binary(y_val, y_pred)
#         cv_scores.append(score)

#     cv_score = np.mean(cv_scores)
#     print('cv_score (Custom metric for target 1): ', cv_score)

#     return cv_score

# # Create an Optuna study and optimize
# study = optuna.create_study(direction='maximize')
# with tqdm(total=100, desc="Optimization Progress") as pbar:
#     study.optimize(lgb_objective, n_trials=100, callbacks=[lambda study, trial: pbar.update(1)])

# # Print the best parameters and best score
# print("Best parameters: ", study.best_params)
# print("Best custom metric score for target 1: ", study.best_value)

[I 2024-09-11 14:01:10,254] A new study created in memory with name: no-name-e7b3cf98-edc4-4c12-a7a7-f5a8bf01381a
Optimization Progress:   1%|          | 1/100 [00:03<05:02,  3.05s/it]

cv_score (Custom metric for target 1):  0.16605767858314063


[I 2024-09-11 14:01:14,626] Trial 1 finished with value: 0.1644187714472077 and parameters: {'lambda_l1': 0.6287850735673677, 'lambda_l2': 0.1666748811080456, 'learning_rate': 0.03544952207815536, 'max_depth': 6, 'num_leaves': 125, 'colsample_bytree': 0.8978148188914179, 'colsample_bynode': 0.5907400487171054, 'bagging_fraction': 0.4378906629467778, 'bagging_freq': 1, 'min_child_samples': 79, 'scale_pos_weight': 2.1342742750782735}. Best is trial 0 with value: 0.16605767858314063.
Optimization Progress:   2%|▏         | 2/100 [00:04<03:18,  2.03s/it]

cv_score (Custom metric for target 1):  0.1644187714472077


[I 2024-09-11 14:01:15,705] Trial 2 finished with value: 0.16491199238563453 and parameters: {'lambda_l1': 1.8916021554552496, 'lambda_l2': 2.2511849354022133, 'learning_rate': 0.04094913615668803, 'max_depth': 5, 'num_leaves': 57, 'colsample_bytree': 0.45872586827244705, 'colsample_bynode': 0.9013574104777118, 'bagging_fraction': 0.6978771291433414, 'bagging_freq': 4, 'min_child_samples': 24, 'scale_pos_weight': 1.8889398042905214}. Best is trial 0 with value: 0.16605767858314063.
Optimization Progress:   3%|▎         | 3/100 [00:05<02:34,  1.59s/it]

...

cv_score (Custom metric for target 1):  0.16542271920677645


[I 2024-09-11 14:04:04,068] Trial 98 finished with value: 0.16503277735327154 and parameters: {'lambda_l1': 0.03470148284833235, 'lambda_l2': 6.316941643814865, 'learning_rate': 0.07065335656898544, 'max_depth': 7, 'num_leaves': 87, 'colsample_bytree': 0.7007110272426774, 'colsample_bynode': 0.7274094655917613, 'bagging_fraction': 0.6118396582647361, 'bagging_freq': 7, 'min_child_samples': 21, 'scale_pos_weight': 3.041168397128955}. Best is trial 79 with value: 0.1678834053526776.
Optimization Progress:  99%|█████████▉| 99/100 [02:53<00:02,  2.31s/it]

cv_score (Custom metric for target 1):  0.16503277735327154


[I 2024-09-11 14:04:05,989] Trial 99 finished with value: 0.1634872987098194 and parameters: {'lambda_l1': 0.0151741984778353, 'lambda_l2': 1.5044630248289221, 'learning_rate': 0.06380931947361813, 'max_depth': 6, 'num_leaves': 130, 'colsample_bytree': 0.6817289572584067, 'colsample_bynode': 0.569835683230459, 'bagging_fraction': 0.8008673103591412, 'bagging_freq': 7, 'min_child_samples': 11, 'scale_pos_weight': 1.745661936570333}. Best is trial 79 with value: 0.1678834053526776.
Optimization Progress: 100%|██████████| 100/100 [02:55<00:00,  1.76s/it]

cv_score (Custom metric for target 1):  0.1634872987098194
Best parameters:  {'lambda_l1': 0.019896528655492955, 'lambda_l2': 2.2749018975450874, 'learning_rate': 0.06290066481867992, 'max_depth': 7, 'num_leaves': 172, 'colsample_bytree': 0.5781905414616991, 'colsample_bynode': 0.46898265874724626, 'bagging_fraction': 0.8137092082350226, 'bagging_freq': 7, 'min_child_samples': 65, 'scale_pos_weight': 3.028819756924015}
Best custom metric score for target 1:  0.1678834053526776


Best custom metric score for target 1:  0.1678834053526776

In [18]:
lgb_params = {
        'verbosity':        -1,
        'objective': 'binary',
        'num_class': 1,
        'boosting_type':    'gbdt',
        'n_estimators': 200,
         'lambda_l1': 0.019896528655492955,
        'lambda_l2': 2.2749018975450874,
        'learning_rate': 0.06290066481867992,
        'max_depth': 7,
        'num_leaves': 172,
        'colsample_bytree': 0.5781905414616991,
        'colsample_bynode': 0.46898265874724626,
        'bagging_fraction': 0.8137092082350226,
        'bagging_freq': 7,
        'min_child_samples': 65,
        'scale_pos_weight': 3.028819756924015,
        'device':'cpu'}

lgb_model = Pipeline([
    ('classifier', lgb.LGBMClassifier(**lgb_params, random_state=42)),
])

<h2> CAT Hypertuning

In [16]:
# def cb_objective(trial):
#     params = {
#         'loss_function': 'Logloss',
#         'iterations': 200,
#         'verbose': False,
#         'task_type': 'CPU',
#         'max_depth': trial.suggest_int('max_depth', 4, 8),
#         'learning_rate': trial.suggest_float('learning_rate', 1e-2, 1e-1, log=True),
#         'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
#         'subsample': trial.suggest_float('subsample', 0.4, 1.0),
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 100),
#         'scale_pos_weight' : trial.suggest_float('scale_pos_weight', 0.8, 4.0),
#         'bootstrap_type': 'Bernoulli',
#     }

#     X = sampled_df[feature_cols]
#     y = sampled_df[target_col]
#     fold = sampled_df['fold']

#     ps = PredefinedSplit(fold)

#     cv_scores = []
#     for train_index, val_index in ps.split():
#         X_train, X_val = X.iloc[train_index], X.iloc[val_index]
#         y_train, y_val = y.iloc[train_index], y.iloc[val_index]

#         model = CatBoostClassifier(**params, random_seed=42)
#         model.fit(X_train, y_train)

#         y_pred = model.predict_proba(X_val)[:, 1]  #
#         score = custom_metric_binary(y_val, y_pred)
#         cv_scores.append(score)

#     cv_score = np.mean(cv_scores)
#     print('cv_score (Custom metric for binary classification): ', cv_score)

#     return cv_score


# study = optuna.create_study(direction='maximize')
# n_trials = 100

# with tqdm(total=n_trials, desc="Optimization Progress") as pbar:
#     def tqdm_callback(study, trial):
#         pbar.update(1)
#     study.optimize(cb_objective, n_trials=n_trials, callbacks=[tqdm_callback])

# print("Best parameters: ", study.best_params)
# print("Best custom metric score for target 1: ", study.best_value)

[I 2024-09-11 14:04:06,020] A new study created in memory with name: no-name-6fc3409a-ff8b-416b-8d94-c7d5739d20ab
Optimization Progress:   1%|          | 1/100 [00:03<05:41,  3.45s/it]

cv_score (Custom metric for binary classification):  0.1496882912688836


[I 2024-09-11 14:04:13,210] Trial 1 finished with value: 0.15492825219501816 and parameters: {'max_depth': 5, 'learning_rate': 0.011883891061326058, 'l2_leaf_reg': 0.4695462642494762, 'subsample': 0.9529457717550645, 'min_data_in_leaf': 71, 'scale_pos_weight': 3.0801383606264396}. Best is trial 1 with value: 0.15492825219501816.
Optimization Progress:   2%|▏         | 2/100 [00:07<05:54,  3.62s/it]

cv_score (Custom metric for binary classification):  0.15492825219501816


[I 2024-09-11 14:04:24,182] Trial 2 finished with value: 0.1569980488734138 and parameters: {'max_depth': 7, 'learning_rate': 0.02189329830723078, 'l2_leaf_reg': 0.012563871015927874, 'subsample': 0.8308990428727852, 'min_data_in_leaf': 100, 'scale_pos_weight': 1.4671881477037498}. Best is trial 2 with value: 0.1569980488734138.
Optimization Progress:   3%|▎         | 3/100 [00:18<11:16,  6.98s/it]

...

cv_score (Custom metric for binary classification):  0.16304999295610167


[I 2024-09-11 14:11:59,846] Trial 98 finished with value: 0.16614108059041355 and parameters: {'max_depth': 5, 'learning_rate': 0.07466914505150628, 'l2_leaf_reg': 8.539305470329156, 'subsample': 0.8984175664471633, 'min_data_in_leaf': 36, 'scale_pos_weight': 3.668365584570356}. Best is trial 98 with value: 0.16614108059041355.
Optimization Progress:  99%|█████████▉| 99/100 [07:53<00:04,  4.21s/it]

cv_score (Custom metric for binary classification):  0.16614108059041355


[I 2024-09-11 14:12:03,350] Trial 99 finished with value: 0.16595790605620447 and parameters: {'max_depth': 5, 'learning_rate': 0.06691966056229726, 'l2_leaf_reg': 8.241958091767698, 'subsample': 0.8952373584982409, 'min_data_in_leaf': 32, 'scale_pos_weight': 3.6539472530429764}. Best is trial 98 with value: 0.16614108059041355.
Optimization Progress: 100%|██████████| 100/100 [07:57<00:00,  4.77s/it]

cv_score (Custom metric for binary classification):  0.16595790605620447
Best parameters:  {'max_depth': 5, 'learning_rate': 0.07466914505150628, 'l2_leaf_reg': 8.539305470329156, 'subsample': 0.8984175664471633, 'min_data_in_leaf': 36, 'scale_pos_weight': 3.668365584570356}
Best custom metric score for target 1:  0.16614108059041355


Best custom metric score for target 1:  0.16614108059041355

In [22]:
cb_params = {
            'loss_function': 'Logloss',
             'verbose':           False,
            'iterations': 200,
            'max_depth': 5,
            'learning_rate': 0.07466914505150628,
            'l2_leaf_reg': 8.539305470329156,
            'subsample': 0.8984175664471633,
            'min_data_in_leaf': 36,
            'scale_pos_weight': 3.668365584570356,
             'bootstrap_type': 'Bernoulli',
             'task_type': 'CPU',
            }

cb_model = Pipeline([
    ('classifier', cb.CatBoostClassifier(**cb_params, random_seed=42)),
])

<h2> XGB Hypertuning

In [17]:
# def xgb_objective(trial):
#     params = {
#         'objective': 'binary:logistic',
#         'enable_categorical': True,
#         'tree_method': 'hist',
#         'random_state': 42,
#         'learning_rate': trial.suggest_float('learning_rate', 1e-2, 1e-1, log=True),
#         'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log=True),
#         'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True),
#         'max_depth': trial.suggest_int('max_depth', 3, 10),
#         'subsample': trial.suggest_float('subsample', 0.5, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
#         'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1.0),
#         'colsample_bynode': trial.suggest_float('colsample_bynode', 0.5, 1.0),
#         'scale_pos_weight' : trial.suggest_float('scale_pos_weight', 0.8, 4.0),
#         'n_estimators': 200,
#     }

#     X = sampled_df[feature_cols]
#     y = sampled_df[target_col]
#     fold = sampled_df['fold']

#     ps = PredefinedSplit(fold)

#     cv_scores = []
#     for train_index, val_index in ps.split():
#         X_train, X_val = X.iloc[train_index], X.iloc[val_index]
#         y_train, y_val = y.iloc[train_index], y.iloc[val_index]

#         model = xgb.XGBClassifier(**params)
#         model.fit(X_train, y_train)

#         y_pred = model.predict_proba(X_val)[:, 1]
#         score = custom_metric_binary(y_val, y_pred)
#         cv_scores.append(score)

#     cv_score = np.mean(cv_scores)
#     print('cv_score (Custom metric for binary classification): ', cv_score)

#     return cv_score


# study = optuna.create_study(direction='maximize')
# n_trials = 100


# with tqdm(total=n_trials, desc="Optimization Progress") as pbar:
#     def tqdm_callback(study, trial):
#         pbar.update(1)
#     study.optimize(xgb_objective, n_trials=n_trials, callbacks=[tqdm_callback])


# print("Best parameters: ", study.best_params)
# print("Best custom metric score for binary classification: ", study.best_value)

[I 2024-09-11 14:12:03,394] A new study created in memory with name: no-name-b497fe26-91a6-414a-b9c5-bb699fdfae58
Optimization Progress:   1%|          | 1/100 [00:01<01:50,  1.12s/it]

cv_score (Custom metric for binary classification):  0.16225829976274722


[I 2024-09-11 14:12:07,516] Trial 1 finished with value: 0.16602725794803627 and parameters: {'learning_rate': 0.047996921980471924, 'lambda': 0.019633362667907807, 'alpha': 1.0918032757189957, 'max_depth': 8, 'subsample': 0.6097324629281844, 'colsample_bytree': 0.953140357508534, 'colsample_bylevel': 0.796621360341649, 'colsample_bynode': 0.8998757781082074, 'scale_pos_weight': 2.996643020729401}. Best is trial 1 with value: 0.16602725794803627.
Optimization Progress:   2%|▏         | 2/100 [00:04<03:38,  2.22s/it]

cv_score (Custom metric for binary classification):  0.16602725794803627


[I 2024-09-11 14:12:12,419] Trial 2 finished with value: 0.1639178941627724 and parameters: {'learning_rate': 0.0243151944427362, 'lambda': 0.22032580512733987, 'alpha': 5.299640801579381, 'max_depth': 10, 'subsample': 0.8764654951750285, 'colsample_bytree': 0.5748164791517535, 'colsample_bylevel': 0.6465057224305273, 'colsample_bynode': 0.7886614442317108, 'scale_pos_weight': 2.8308489387343014}. Best is trial 1 with value: 0.16602725794803627.
Optimization Progress:   3%|▎         | 3/100 [00:09<05:34,  3.45s/it]

...

cv_score (Custom metric for binary classification):  0.16501496176165784


[I 2024-09-11 14:16:58,281] Trial 98 finished with value: 0.16680572689217515 and parameters: {'learning_rate': 0.08002681685808082, 'lambda': 1.1680718454197951, 'alpha': 0.0016655844738438861, 'max_depth': 7, 'subsample': 0.539202230686883, 'colsample_bytree': 0.8064018889800982, 'colsample_bylevel': 0.9144971808824292, 'colsample_bynode': 0.59052133533059, 'scale_pos_weight': 2.1124806027359226}. Best is trial 68 with value: 0.1680480591798318.
Optimization Progress:  99%|█████████▉| 99/100 [04:54<00:03,  3.06s/it]

cv_score (Custom metric for binary classification):  0.16680572689217515


[I 2024-09-11 14:17:01,015] Trial 99 finished with value: 0.16813904672815277 and parameters: {'learning_rate': 0.05566033566180181, 'lambda': 4.62951764198257, 'alpha': 0.19502364654812793, 'max_depth': 8, 'subsample': 0.726958618773603, 'colsample_bytree': 0.884578248658937, 'colsample_bylevel': 0.7790735100475025, 'colsample_bynode': 0.7638419540477621, 'scale_pos_weight': 2.5112778372226665}. Best is trial 99 with value: 0.16813904672815277.
Optimization Progress: 100%|██████████| 100/100 [04:57<00:00,  2.98s/it]

cv_score (Custom metric for binary classification):  0.16813904672815277
Best parameters:  {'learning_rate': 0.05566033566180181, 'lambda': 4.62951764198257, 'alpha': 0.19502364654812793, 'max_depth': 8, 'subsample': 0.726958618773603, 'colsample_bytree': 0.884578248658937, 'colsample_bylevel': 0.7790735100475025, 'colsample_bynode': 0.7638419540477621, 'scale_pos_weight': 2.5112778372226665}
Best custom metric score for binary classification:  0.16813904672815277


Best custom metric score for binary classification:  0.16813904672815277

In [23]:
xgb_params = {
            'objective': 'binary:logistic',
            'enable_categorical': True,
            'tree_method':        'hist',
            'random_state':       42,
            'learning_rate': 0.05566033566180181,
            'lambda': 4.62951764198257,
            'alpha': 0.19502364654812793,
            'max_depth': 8,
            'subsample': 0.726958618773603,
            'colsample_bytree': 0.884578248658937,
            'colsample_bylevel': 0.7790735100475025,
            'colsample_bynode': 0.7638419540477621,
            'scale_pos_weight': 2.5112778372226665,
             'n_estimators': 200}

xgb_model = Pipeline([
    ('classifier', xgb.XGBClassifier(**xgb_params)),
])

In [24]:
estimator = VotingClassifier([
    ('lgb', lgb_model), ('cb', cb_model), ('xgb', xgb_model),
], voting='soft')

In [25]:
from sklearn.metrics import roc_auc_score, make_scorer

custom_scorer = make_scorer(custom_metric_binary, needs_proba=True, greater_is_better=True)

X = sampled_df[feature_cols]
y = sampled_df[target_col]
fold = sampled_df['fold']
ps = PredefinedSplit(fold)

val_score = cross_val_score(estimator, X, y, cv=ps, scoring=custom_scorer)

print(np.mean(val_score), val_score)

0.1688309231705795 [0.17298682 0.17352485 0.158034   0.16182014 0.1777888 ]
